# Explore 'old' dataset

### Import packages

In [1]:
# Import packages
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mne

sns.set_theme(style="whitegrid")
%matplotlib inline
# %matplotlib qt

In [2]:
import fnmatch
import warnings
import re
warnings.filterwarnings('ignore')

import helper_functions

In [3]:
#from config_for_repro_prepro  import PATH_RAW_DATA, PATH_METADATA, PATH_DATA_PROCESSED_ML

### Now let's get all our old cnt files together

In [4]:
dir_names = {11: '11mnd mmn',
             17: '17mnd mmn',
             23: '23mnd mmn',
             29: '29mnd mmn',
             35: '35mnd mmn',
             41: '41mnd mmn',
             47: '47mnd mmn'}
    
df_list = []

for age_group, directory in dir_names.items(): # Go into every age group folder        
    dir_path = os.path.join('C:/Projects/EEG_explorer/Data', directory)
    file_names = os.listdir(dir_path)
    
    cnt_paths = [os.path.join(dir_path, file_name) for file_name in fnmatch.filter(file_names, "*.cnt")]
    # list comprehension - creates a list of all cnt file names with thisdirectory
    cnt_files = [os.path.basename(x)[:-4] for x in cnt_paths]
    # list comprehension
    codes = [int(re.search(r'\d+', x).group()) for x in cnt_files]
    # takes number out of string (\d+ takes out digits)
    df = pd.DataFrame(list(zip(codes, cnt_paths, cnt_files)), columns=['code', 'cnt_path','cnt_file']) 
    
    df['age_group'] = age_group
    df_list.append(df)

cnt_files = pd.concat(df_list)

In [24]:
print( "We have", len(cnt_files), ".cnt files")

We have 2149 .cnt files


### Now let's get all our old edf files together

In [16]:
dir_names = {11: '11mnd mmn',
             17: '17mnd mmn',
             23: '23mnd mmn',
             29: '29mnd mmn',
             35: '35mnd mmn',
             41: '41mnd mmn',
             47: '47mnd mmn'}
    
edf_df_list = []

for age_group, directory in dir_names.items(): # Go into every age group folder        
    dir_path = os.path.join('C:/Projects/EEG_explorer/Data', directory)
    file_names = os.listdir(dir_path)
    
    edf_paths = [os.path.join(dir_path, file_name) for file_name in fnmatch.filter(file_names, "*.edf")]
    # list comprehension - creates a list of all cnt file names with thisdirectory
    edf_files = [os.path.basename(x)[:-4] for x in edf_paths]
    # list comprehension
    #codes = [int(re.search(r'\d+', x).group()) for x in cnt_files]
    # takes number out of string (\d+ takes out digits)
    df_edf = pd.DataFrame(list(zip(edf_paths, edf_files)), columns=['edf_path','edf_file']) 
    
    
    edf_df_list.append(df_edf)

edf_files = pd.concat(edf_df_list)

In [25]:
print( "We have", len(edf_files), ".edf files")

We have 157 .edf files


### We can now clean out the "cleaned", but not be sure about certain other changes

In [26]:
# make set of all files,,
# make sure of probably cleaned files

# set of all files
files_list = []
cleaned_files_list = []
for i in cnt_files['cnt_path']:
    files_list.append(i)
    if 'cleaned' in i:
        print(i)
if len(cleaned_files_list) == 0:
    print("No .cnt files have cleaned in the name")
    


No .cnt files have cleaned in the name


In [28]:
edf_files_list = []
edf_cleaned_files_list = []
for i in edf_files['edf_path']:
    edf_files_list.append(i)
    if 'cleaned' in i:
        edf_cleaned_files_list.append(i)
if len(edf_cleaned_files_list) == 0:
    print("No .edf files have cleaned in the name")
else:
    print(len(edf_cleaned_files_list), "files appear to have been cleaned or have it in the name, out of", len(edf_files_list) )


157 files appear to have been cleaned or have it in the name, out of 157


### So we can see the edf files are cleaned, but were they just preprocessed, or what process generated them?

## Let's also examine the metadata: